In [67]:
import pandas as pd
import numpy as np


In [68]:
data = pd.read_csv("gemstone.csv")

In [69]:
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [70]:
data.drop('id', axis=1, inplace=True)

In [71]:
X = data.drop('price', axis=1)
y = data[['price']]

In [72]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [73]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [74]:
cat_cols = X.select_dtypes(include=object).columns
num_cols = X.select_dtypes(exclude=object).columns

In [75]:
cat_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [76]:
num_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [77]:
##Defining the Categories ranking

In [78]:
cut_category = ['Fair','Good','Very Good', 'Premium','Ideal']
color_category = ['D','E','F','G','H','I','J']
calrity_category = ['I1','SI2','SI1','VS2', 'VS1','VVS2','VVS1','IF']

In [79]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [80]:
num_pipeline = Pipeline(
steps=[
   ('imputer', SimpleImputer()),
   ('scaler', StandardScaler())
]
)

In [81]:
cat_pipeline = Pipeline(
steps=[
   ('imputer', SimpleImputer(strategy='most_frequent')),
   ('scaler', OrdinalEncoder(categories= [cut_category,color_category,calrity_category]))
]
)

In [82]:
preprosses = ColumnTransformer(
    [
        ('numPipeline',num_pipeline,num_cols),
        ('catPipeline',cat_pipeline, cat_cols)
    ]
)

In [83]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30, random_state= 30)

In [84]:
preprosses.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  4.        ,
         5.        ,  5.        ],
       [ 0.2351953 ,  1.83363716, -0.12153081, ...,  1.        ,
         1.        ,  2.        ],
       [ 0.49461699,  0.81585507,  0.39980029, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ...,  1.        ,
         3.        ,  2.        ],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  4.        ,
         3.        ,  2.        ]])

In [85]:
preprosses.transform(X_test)

array([[-0.56468825, -0.94213218, -0.6428619 , ...,  3.        ,
         1.        ,  3.        ],
       [-0.17555571,  1.00090636, -0.12153081, ...,  2.        ,
         4.        ,  2.        ],
       [-1.06191316,  0.2607012 , -0.12153081, ...,  4.        ,
         4.        ,  7.        ],
       ...,
       [-0.19717419, -3.34779894,  1.44246248, ...,  3.        ,
         6.        ,  3.        ],
       [-0.82410994, -0.20192702, -0.12153081, ...,  4.        ,
         3.        ,  2.        ],
       [ 2.61322747, -0.75708089,  1.44246248, ...,  3.        ,
         6.        ,  3.        ]])

In [86]:
preprosses.get_feature_names_out()

array(['numPipeline__carat', 'numPipeline__depth', 'numPipeline__table',
       'numPipeline__x', 'numPipeline__y', 'numPipeline__z',
       'catPipeline__cut', 'catPipeline__color', 'catPipeline__clarity'],
      dtype=object)

In [87]:
X_train = pd.DataFrame(preprosses.fit_transform(X_train),columns= preprosses.get_feature_names_out())
X_test = pd.DataFrame(preprosses.transform(X_test),columns=preprosses.get_feature_names_out())

In [88]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

In [89]:
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [108]:
models = {
    'linear':LinearRegression(),
    'rigde': Ridge(),
    'lasso': Lasso(),
    'elastic': ElasticNet()
}

In [109]:
import numpy as np
def evalute_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae,rmse,r2_square


In [110]:
models_list = []

In [111]:
for i in range(len(list((models)))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    #PRediction

    y_pred = model.predict(X_test)
    mae,rmse, r2_square = evalute_model(y_test,y_pred)   
    print(list(models.keys())[i])
    models_list.append(list(models.keys())[i])
    print("Model Training Performance")
    print('MAE',mae)
    print('rmse',rmse)
    print('r2_score',r2_square*100)
    print('*'*35)

linear
Model Training Performance
MAE 674.0255115796833
rmse 1013.9047094344004
r2_score 93.68908248567512
***********************************
rigde
Model Training Performance
MAE 674.0565132295983
rmse 1013.9058997761055
r2_score 93.6890676674133
***********************************
lasso
Model Training Performance
MAE 675.2036880701621
rmse 1013.8723151049944
r2_score 93.68948574778251
***********************************
elastic
Model Training Performance
MAE 1050.7468664314322
rmse 1513.914035022976
r2_score 85.92978759337907
***********************************
